# 데이터 전처리

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
# 현재 폴더 경로 확인
os.getcwd()

In [ ]:
# 데이터 폴더로 이동
os.chdir('../../data')

In [ ]:
# 현재 경로 폴더의 파일 조회
sorted(os.listdir())

In [ ]:
# 엑셀 데이터 불러오기
apt = pd.read_excel('APT_Price_Seoul_Merged_2020~2024.xlsx')

In [ ]:
apt.head()

In [ ]:
apt.info()

In [ ]:
# 코드 오른쪽 끝에 \ 를 추가하면 아래 줄 코드를 연결하라는 의미
# \ 오른쪽에 아무것도 추가하면 안됨
apt = pd.read_csv('APT_Price_Seoul_Merged_2020~2024.csv', 
                low_memory=False,
                parse_dates=['등기일자']
                )

In [ ]:
apt.info()

In [ ]:
apt['지역코드'] # 생략
apt.loc[:, '지역코드'] # 원형

In [ ]:
apt[['지역코드', '거래금액']] # 생략
apt.loc[:, ['지역코드', '거래금액']] # 원형

In [ ]:
apt.loc[:, ['거래금액', '지역코드', '아파트']]
# apt['거래금액':'아파트'] # 에러 발생
apt.loc[:, '거래금액':'아파트']

In [ ]:
apt.head()

In [ ]:
apt.loc[:, ['거래금액', '지역코드', '건축년도', '년', '월', '일', '층']]

In [ ]:
apt.columns

In [ ]:
apt.dtypes == 'int64'

### 컬럼 타입에 따라 슬라이싱

In [ ]:
apt.loc[:, apt.dtypes == 'int64']

In [ ]:
apt.loc[:, (apt.dtypes == 'int64') | (apt.dtypes == 'float64')]

In [ ]:
apt.select_dtypes(include=[int, float])

In [ ]:
# 데이터프레임의 열이름은 인덱스 자료형
apt.columns

In [ ]:
# str accessor(접근자)는 문자열 시리즈 또는 인덱스 자료형에서 문자열을 처리하는 함수를 다수 포함하고 있는 특수 객체(모듈)
apt.columns.str.contains(pat='년')

In [ ]:
# 파이프(|)는 정규 표현식에서 or를 의미하는 메타문자
apt.loc[:, apt.columns.str.contains(pat='년|월|일')]

### 정규표현식

In [ ]:
nums = pd.Series(data=['M:010-2345-6789', 'T:02-345-6789', 'ID:001231-7890123'])

nums.str[0]
# 0    M
# 1    T
# 2    I
# dtype: object

nums.str[0:2]
# 0    M:
# 1    T:
# 2    ID
# dtype: object

In [ ]:
nums.str.extract(pat='(M|T|ID)')
nums.str.extract(pat='([A-Z]+)')
#   0
# 0	M
# 1	T
# 2	ID

In [ ]:
nums.str.split(pat=':')
# 0      [M, 010-2345-6789]
# 1        [T, 02-345-6789]
# 2    [ID, 001231-7890123]
# dtype: object

In [ ]:
nums.str.split(pat=':|-')
# 0     [M, 010, 2345, 6789]
# 1       [T, 02, 345, 6789]
# 2    [ID, 001231, 7890123]
# dtype: object

In [ ]:
# 원소에 없는 공백(' ')을 추가하면 분리X
# ''안에 공백또한 패턴으로 인식
nums.str.split(pat=':|-').str[0]
# 0     M
# 1     T
# 2    ID
# dtype: object

In [ ]:
nums.str.split(pat=':|-', expand=True)
#   0	1	    2	    3
# 0	M	010	    2345	6789
# 1	T	02	    345	    6789
# 2	ID	001231	7890123	None


In [ ]:
nums.str.replace(pat='-', repl=' ')

In [ ]:
# str.extract는 실행 결과로 항상 데이터프레임을 반환
nums.str.extract(pat='([0-9]+)')
# 0
# 0	010
# 1	02
# 2	001231

In [ ]:
# ^(캐럿) : 문자열 시작 위치를 지정하는 앵커
nums.str.extract(pat='(^[0-9]+)')
#   0
# 0	NaN
# 1	NaN
# 2	NaN

In [ ]:
# $ : 문자열 끝 위치를 지정하는 앵커
nums.str.extract(pat='([0-9]+$)')
#   0
# 0	6789
# 1	6789
# 2	7890123

In [ ]:
# 전화번호의 가운데에 있는 숫자만 추출
nums.str.extract(pat='(-[0-9]+-)')[0].str.replace(pat='-', repl='')
# 0    2345
# 1     345
# 2     NaN
# Name: 0, dtype: object

In [ ]:
# (?<=-) : 후방탐색, 하이픈 다음에 오는 문자열을 추출하고, 하이픈은 제거
# (?=-) : 전방탐색, 하이픈 앞에 오는 문자열을 추출하고, 하이픈 제거
nums.str.extract(pat='((?<=-)[0-9]+(?=-))')[0]

### 조건에 맞는 행 선택 : 연속형

In [ ]:
cond = apt['거래금액'] >= 1500000
apt.loc[cond, :]

In [ ]:
cond = (apt['거래금액'] <= 1500000) & (apt['층'] >= 65)
apt.loc[cond, :]

### 시리즈의 비교 연산 메서드

In [ ]:
cond = apt['거래금액'].lt(1500000) & apt['층'].ge(65)
apt.loc[cond, :]

In [ ]:
# 파이썬에서 천 단위 구분자로 언더스코어를 추가할 수 있음
1_500_000 # 1500000

In [ ]:
cond = apt['층'].ge(10) & apt['층'].le(20)
apt.loc[cond, :].shape # (88699, 15)

cond = apt['층'].between(10, 20)
apt.loc[cond, :].shape # (88699, 15)

### 조건에 맞는 행 선택 : 범주형

In [ ]:
cond = apt['시군구'].eq('강남구')
apt.loc[cond, :].shape # (12790, 15)

In [ ]:
cond = apt['시군구'].eq('강남구') | apt['시군구'].eq('서초구')
apt.loc[cond, :].shape # (23526, 15)

cond_1 = apt['시군구'].eq('강남구')
cond_2 = apt['시군구'].eq('서초구')
apt.loc[cond_1 | cond_2, :].shape # (23526, 15)

In [ ]:
cond = apt['시군구'].eq('강남구') | apt['시군구'].eq('서초구') | apt['시군구'].eq('송파구')
apt.loc[cond, :].shape # (37970, 15)

cond_1 = apt['시군구'].eq('강남구')
cond_2 = apt['시군구'].eq('서초구')
cond_3 = apt['시군구'].eq('송파구')
apt.loc[cond_1 | cond_2 | cond_3, :].shape # (37970, 15)

In [ ]:
cond = apt['시군구'].isin(['강남구', '서초구', '송파구'])
apt.loc[cond, :].shape # (37970, 15)

In [ ]:
cond = apt['시군구'].str.contains(pat='^강')
apt.loc[cond, :].shape # (44193, 15)

In [ ]:
# 강남구 청담동 거래 건수
cond1 = apt['시군구'].eq('강남구')
cond2 = apt['법정동'].eq('청담동')
apt.loc[cond1 & cond2, :].shape

In [ ]:
# 강남구 청담동 PH129
cond1 = apt['시군구'].eq('강남구')
cond2 = apt['법정동'].eq('청담동')
cond3 = apt['아파트'].eq('PH129')
apt.loc[cond1 & cond2 & cond3, :].shape

### 열이름으로 열 삭제

In [ ]:
apt = apt.drop(columns=['지역코드', '매도자'])

In [ ]:
apt.head()

### 행이름으로 행 삭제

In [ ]:
apt1 = apt.sample(5, random_state=1234)
apt1

In [ ]:
# drop 메서드는 눈에 보이는 행이름으로 행을 삭제
apt1.drop(index=[163874, 220404])

In [ ]:
# 데이터프레임에서 처음 두 행의 인덱스를 슬라이싱
apt1.index[0:2]

In [ ]:
# 데이터프레임의 눈에 보이는 행이름에 관계없이 처음 두 행을 삭제
apt1.drop(index=apt1.index[0:2])

### 논리연산자 vs 비트연산자

- 논리 연산자는 True(스칼라) 또는 False(스칼라) 사이에 사용됨
- 비트 연산자는 True 또는 False를 원소로 갖는 배열, 시리즈 사이에서 사용됨

In [ ]:
sr = pd.Series([1,2,3])

In [ ]:
# all() 메서드는 모든 원소가 True일 때 True, 나머지는 False
# any() 메서드는 하나 이상의 원소가 True일 때 True, 나머지는 False
(sr == 1).all() # np.False_
(sr == 1).any() # np.True_

In [ ]:
(sr == 1) and (sr == 3)
# ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

(sr == 1).any() and (sr == 3).any()
# np.True_

### 열이름 변경

In [ ]:
apt.rename(columns={
    '아파트': '단지명',
    '건축년도': '입주년도'
})

In [ ]:
apt.columns

In [ ]:
apt.columns = ['거래금액', '단지명', '시도명', '시군구', '법정동', '지번', '입주년도', '계약년도', '계약월', '계약일', '등기일자',
        '전용면적', '층']

In [ ]:
apt.head()

### 행이름 변경

In [ ]:
df1 = apt1

In [ ]:
df2 = df1.set_index(['아파트', '시도명', '시군구', '법정동', '지번'])
df2

In [ ]:
df1.reset_index(drop=True)

In [ ]:
df2 = df2.reset_index()
df2

In [ ]:
df2

### 파생변수 생성 : 연속형

In [ ]:
# 시리즈 또는 데이터프레임에서 실수형 원소 또는 셀 값을 반올림하여 소수점 둘째자리까지 표현
# 원본 데이터는 유지
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [ ]:
apt['평당금액'] = apt['거래금액'] / apt['전용면적'] * 3.3
apt.head()

In [ ]:
apt['거래금액'] = apt['거래금액'] / 10000
apt.head()

In [ ]:
apt['경과년수'] = apt['계약년도'] - apt['입주년도']
apt.head()

### 구간화

In [ ]:
# for문으로 구간화
imsi = apt.copy()

result = []
for i in apt['경과년수']:
    if i >= 30:
        result.append('충족')
    else:
        result.append('부족')

imsi = imsi.assign(재건축 = result)
imsi.head()

In [ ]:
cond = apt['경과년수'].ge(30)
apt['재건축'] = np.where(cond, '충족', '부족')
apt.head()

### 구간화 함수

In [ ]:
np.select(
    condlist=[
        apt['경과년수'].le(5),
        apt['경과년수'].le(10),
        apt['경과년수'].gt(10)
    ],
    choicelist=['신축', '준신축', '구축'],
    default='0'
)

In [ ]:
apt.head()

In [ ]:
# 전용면적이 80이하면 소형, 80초과 100 이하면 중형, 100 초과 200이하면 대형 200초과면 초대형
np.select(
    condlist=[
        apt['전용면적'].le(80),
        apt['전용면적'].le(100),
        apt['전용면적'].le(200),
        apt['전용면적'].gt(200),
    ],
    choicelist=['소형', '중형', '대형', '초대형'],
    default='0'
)

In [ ]:
# np.where 함수에 조건만 지정하면 True인 인덱스를 튜플로 반환
# 조건을 만족하는 인덱스 배열을 재사용하려면 [0] 인덱싱 필요
cond = apt['경과년수'].ge(30)
np.where(cond)[0]

In [ ]:
cond = apt['경과년수'].ge(30)
apt['재건축'] = np.where(cond, '충족', '부족')
apt['재건축']
# 0         부족
# 1         부족
# 2         충족
# 3         부족
# 4         충족
#           ..
# 231899    부족
# 231900    부족
# 231901    부족
# 231902    부족
# 231903    부족
# Name: 재건축, Length: 231904, dtype: object

In [ ]:
apt['경과년수'].min()
apt['경과년수'].max()

In [ ]:
pd.cut(
    x=apt['경과년수'],
    bins=[-2, 5, 10, 62],
    labels=['신축', '준신축', '구축']
)

In [ ]:
pd.qcut(
    x=apt['경과년수'],
    q=3,
    labels=['신축', '준신축', '구축']
)

### 열별 자료형 변환

In [ ]:
apt.dtypes
# 거래금액           float64
# 단지명             object
# 시도명             object
# 시군구             object
# 법정동             object
# 지번              object
# 입주년도             int64
# 계약년도            object
# 계약월             object
# 계약일             object
# 등기일자    datetime64[ns]
# 전용면적           float64
# 층                int64
# 평당금액           float64
# 경과년수             int64
# 재건축             object
# dtype: object

In [ ]:
apt['계약일자'] = apt['계약년도'] + '-' + apt['계약월'] + '-' + apt['계약일']
# UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U1')) -> None

In [ ]:
apt['계약년도'] = apt['계약년도'].astype(str)
apt['계약월'] = apt['계약월'].astype(str)
apt['계약일'] = apt['계약일'].astype(str)
apt.dtypes
# 거래금액           float64
# 단지명             object
# 시도명             object
# 시군구             object
# 법정동             object
# 지번              object
# 입주년도             int64
# 계약년도            object
# 계약월             object
# 계약일             object
# 등기일자    datetime64[ns]
# 전용면적           float64
# 층                int64
# 평당금액           float64
# 경과년수             int64
# 재건축             object
# dtype: object

In [ ]:
cols = ['x1', 'x2', 'x3', 'x4']
apt[cols] = apt[cols].astype(int)